## 8.1 Regular 1D grids

In [ ]:
import pandas as pd
import numpy as np
from netCDF4 import Dataset
import h5py
import scipy.interpolate
from matplotlib import pyplot as plt
from cartopy import crs as ccrs

In [ ]:
fires = pd.read_csv("data/VIIRSNDE_global2018312.v1.0.txt")
fires.head()

In [ ]:
# Number of nx and ny points for the grid. 720 nx, 360 ny creates 1.0 degree grid.
grid_size = 1.0
coverage = [-180.0 , -90.0 , 180.0 , 90.0]

num_points_x = int((coverage[2] - coverage[0])/grid_size)
num_points_y = int((coverage[3] - coverage[1])/grid_size)

print(num_points_x, num_points_y)

In [ ]:
# Using a real step length will skip the end value
print(np.mgrid[0:4:1.0])

# Using a complex number of bins will include the end value
print(np.mgrid[0:4:5j])

In [ ]:
nx = complex(0, num_points_x)
ny = complex(0, num_points_y)

Xnew, Ynew = np.mgrid[coverage[0]:coverage[2]:nx, coverage[1]:coverage[3]:ny]

In [ ]:
fire_count = np.zeros([num_points_x, num_points_y])

for i, lon in enumerate(fires.Lon):
    lat = fires.Lat[i]
    
    adjlat = ((lat + 90) / grid_size);
    adjlon = ((lon + 180 ) / grid_size);
    
    latbin = int(adjlat)
    lonbin = int(adjlon)
    
    fire_count[lonbin, latbin] = fire_count[lonbin, latbin]+1

In [ ]:
fire_count[fire_count == 0] = np.nan

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=[15, 15], \
           subplot_kw={'projection': ccrs.PlateCarree()})

ax1.coastlines()
ax1.set_global()
ax1.scatter(fires['Lon'], fires['Lat'], s=1)

ax2.coastlines()
ax2.set_global()

ax2.pcolormesh(Xnew, Ynew, fire_count, vmin=0, vmax=40)

plt.show()

## 8.2 Regular 2D grids

**Errata:** The book originally imports the operational file, using the path ```data/3B-HHR.MS.MRG.3IMERG.20160811-S233000-E235959.1410.V06B.HDF5```. This path should be changed to  ```data/3B-HHR.MS.MRG.3IMERG.20160811-S233000-E235959.1410.V06B_thinned.HDF5```; this is a smaller version of the operational file. The correct code is shown below.

In [ ]:
fname = 'data/3B-HHR.MS.MRG.3IMERG.20160811-S233000-E235959.1410.V06B_thinned.HDF5'
imergv6 = h5py.File(fname, 'r')

precip = imergv6['Grid/precipitationCal'][0,:,:]
lat = imergv6['Grid/lat'][:]
lon = imergv6['Grid/lon'][:]

In [ ]:
Xold, Yold = np.meshgrid(lon, lat, indexing='ij')

In [ ]:
# Number of nx and ny points for the grid. 720 nx, 360 ny creates 1.0 degree grid.
def create_2d_grid(grid_size):
    coverage = [-180.0 , -90.0 , 180.0 , 90.0]

    num_points_x = int((coverage[2] - coverage[0])/grid_size)
    num_points_y = int((coverage[3] - coverage[1])/grid_size)

    nx = complex(0, num_points_x)
    ny = complex(0, num_points_y)

    Xnew, Ynew = np.mgrid[coverage[0]:coverage[2]:nx, coverage[1]:coverage[3]:ny]
    return Xnew, Ynew

In [ ]:
Xnew, Ynew = create_2d_grid(0.5)

In [ ]:
precip.shape, Yold.shape, Ynew.shape

In [ ]:
values = precip.flatten()

dims = (values.shape[0], 2)
points =  np.zeros(dims)

points[:, 0] = Xold.flatten()
points[:, 1] = Yold.flatten()

In [ ]:
gridOut = scipy.interpolate.griddata(points, values, (Xnew, Ynew), method='nearest')

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=[15, 15], \
   subplot_kw={'projection': ccrs.PlateCarree()})

for ax in axes:
    ax.set_extent([-94.3, -88.8, 28.9, 33.1])
    ax.coastlines('10m', color='orange')

axes[0].set_title("Before regridding")
axes[0].pcolormesh(Xold, Yold, precip, vmin=0, vmax=20)

axes[1].set_title("After regridding")
axes[1].pcolormesh(Xnew, Ynew, gridOut, vmin=0, vmax=20)

plt.show()

## Irregular 2D grids
### Resizing

In [ ]:
fname = "data/goes-meso/michael/OR_ABI-L1b-RadM1-M3C02_G16_s20182822019282_e20182822019339_c20182822019374.nc"
goesnc = Dataset(fname, 'r')
C02 = goesnc.variables['Rad'][:,:]
x = goesnc.variables['x'][:]
y = goesnc.variables['y'][:]

In [ ]:
proj_var = goesnc.variables['goes_imager_projection']

sat_height = proj_var.perspective_point_height
semi_major = proj_var.semi_major_axis
semi_minor = proj_var.semi_minor_axis
central_lon = proj_var.longitude_of_projection_origin
globe = ccrs.Globe(semimajor_axis=semi_major, semiminor_axis=semi_minor)

# Multiply the x, y coordinated by satellite height to get the pixel position
X = x * sat_height
Y = y * sat_height
imgExtent = (X.min(), X.max(), Y.min(), Y.max())

crs = ccrs.Geostationary(central_longitude=central_lon, satellite_height=sat_height, globe=globe)

In [ ]:
imgExtent

In [ ]:
resized_C02 = C02[::20, ::20]

In [ ]:
lims = [0, 500]

fig, axes = plt.subplots(ncols=2,  figsize=[15, 15], \
     subplot_kw={'projection': crs})

for ax in axes:
    ax.set_extent([-90.0, -82.0, 22.0, 30.0])
    ax.coastlines('10m', color='black')

axes[0].set_title("Native Resolution")
im1 = axes[0].imshow(C02, cmap=plt.get_cmap("rainbow"), origin='upper', 
            extent=imgExtent, vmin=lims[0], vmax=lims[1])

axes[1].set_title("20x Lower Resolution")
im2 = axes[1].imshow(resized_C02, cmap=plt.get_cmap("rainbow"), origin='upper', 
            extent=imgExtent, vmin=lims[0], vmax=lims[1])

# Adding colorbars
fig.colorbar(im1, pad=0.05, orientation='horizontal', ax=axes[0])
fig.colorbar(im2, pad=0.05, orientation='horizontal', ax=axes[1])

plt.show()
# plt.savefig('C:\\Users\\rebekah\\Dropbox\\Python book\\AGU_Esmaili_9781119606888_RPP\\c08_rbin_flipped.png')
# plt.close()

### Griddata

In [ ]:
import matplotlib.patches as patches

In [ ]:
fname = "data/goes-meso/michael/OR_ABI-L1b-RadM1-M3C13_G16_s20182822019282_e20182822019350_c20182822019384.nc"
goesnc_sm = Dataset(fname, 'r')
C13 = goesnc_sm.variables['Rad'][:]
sm_x = goesnc_sm.variables['x'][:]
sm_y = goesnc_sm.variables['y'][:]

In [ ]:
vmin=0
vmax=50

delta = 0.005
xbot = -0.025
ybot =  0.075

lower_left = (xbot, ybot)

In [ ]:
fig, axes = plt.subplots(ncols=1, nrows=1, figsize=[10, 10])

meso = axes.pcolormesh(sm_x, sm_y, C13, vmin=vmin, vmax=vmax, cmap=plt.get_cmap("tab20c"))
fig.colorbar(meso, label='Brightness Temperature (K)')
zoom_box = patches.Rectangle(lower_left, delta, delta, 
             linewidth=4, edgecolor='black', facecolor='none')
axes.add_patch(zoom_box)
axes.set_aspect('equal')
axes.axis('off')

plt.show()

In [ ]:
Xold, Yold = np.meshgrid(sm_x, sm_y)
Xnew, Ynew = np.meshgrid(x, y)

In [ ]:
values = C13.flatten()

dims = (values.shape[0], 2)
points =  np.zeros(dims)

points[:, 0] = Xold.flatten()
points[:, 1] = Yold.flatten()

In [ ]:
gridOut_nn = scipy.interpolate.griddata(points, values, (Xnew, Ynew), method='nearest')
gridOut_lin = scipy.interpolate.griddata(points, values, (Xnew, Ynew), method='linear')
gridOut_cube = scipy.interpolate.griddata(points, values, (Xnew, Ynew), method='cubic')

In [ ]:
gridOut_rebin = np.repeat(np.repeat(C13, 4, axis=1), 4, axis=0)

In [ ]:
gridOut = [[gridOut_rebin, gridOut_nn], [gridOut_lin, gridOut_cube]]
labels = [["Re-binning", "Nearest Neighbor"],["Linear", "Cubic"]]
plt.rcParams.update({'font.size': 18, 'figure.figsize': [20, 20]})

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=2)

cmap=plt.get_cmap("tab20c")

for i, axis in enumerate(axes):
    for j, ax in enumerate(axis):
        ax.set_xlim([xbot, xbot+delta])
        ax.set_ylim([ybot, ybot+delta])
        ax.set_aspect('equal')
        ax.set_title(labels[i][j])
        ax.pcolormesh(Xnew, Ynew, gridOut[i][j], vmin=vmin, vmax=vmax, cmap=cmap)
        ax.axis('off')

plt.show()
# plt.savefig('C:\\Users\\rebekah\\Dropbox\\Python book\\AGU_Esmaili_9781119606888_RPP\\c08_nn_compare.png')
# plt.close()

### Resampling

In [ ]:
from pyresample import geometry
from pyresample.kd_tree import resample_nearest

In [ ]:
fname='data/aod/JRR-AOD_v1r1_npp_s201808091955538_e201808091957180_c201808092049460_thinned.nc'
file_id_NPP = Dataset(fname)

aod = file_id_NPP.variables['AOD550'][:,:]
lat = file_id_NPP.variables['Latitude'][:,:]
lon = file_id_NPP.variables['Longitude'][:,:]

In [ ]:
# Input list of swath points
oldLonLat = geometry.SwathDefinition(lons=lon, lats=lat)

In [ ]:
# Create a new grid at 0.1 degree gridding
x = np.arange(lon.min(), lon.max(), 0.1)
y = np.arange(lat.min(), lat.max(), 0.1)
newLon, newLat = np.meshgrid(x, y)

In [ ]:
# define the new grid using 
newLonLat = geometry.GridDefinition(lons=newLon, lats=newLat)

In [ ]:
# Resample the data
newAOD = resample_nearest(oldLonLat, aod, newLonLat, radius_of_influence=5000, fill_value=None)

In [ ]:
lims = [0,1]
fig, axes = plt.subplots(nrows=2,  figsize=[15, 10], \
     subplot_kw={'projection': ccrs.PlateCarree()})

for ax in axes:
    ax.coastlines('10m', color='black')

axes[0].set_title("Before regridding")
axes[0].pcolormesh(lon, lat, aod, vmin=lims[0], vmax=lims[1])

axes[1].set_title("After regridding")
axes[1].pcolormesh(newLon, newLat, newAOD, vmin=lims[0], vmax=lims[1])

plt.show()